In [1]:
from pathlib import Path
from urllib.error import HTTPError
import datetime

from IPython.display import display
import numpy as np
import pandas as pd
import plotly.graph_objects as go

In [2]:
def get_england_deaths_df(date):
    url = f"https://www.england.nhs.uk/statistics/wp-content/uploads/sites/2/2020/04/" \
          f"COVID-19-total-announced-deaths-{date:%-d-%B-%Y}.xlsx"
    
    deaths_df = pd.read_excel(
        url,
        sheet_name="COVID19 total deaths by region",
        header=15,
        index_col=1
    )
    
    deaths_df = deaths_df.dropna(how="all", axis="index").dropna(how="all", axis="columns")
    deaths_df = deaths_df.drop(["Up to 01-Mar-20", "Awaiting verification", "Total"], axis="columns")
    deaths_df = deaths_df.astype(int)
    deaths_df.index = deaths_df.index.rename("area_name").str.strip()
    deaths_df.columns = pd.to_datetime(deaths_df.columns)
    deaths_df = deaths_df.rename(index={
        "East Of England": "East of England", 
        "North East And Yorkshire": "North East and Yorkshire"
    })
        
    deaths_df = deaths_df.cumsum(axis=1)
    
    start_date = datetime.date(2020, 3, 9)
    return deaths_df.loc[:, pd.to_datetime(start_date):]

In [3]:
# deaths data taken from https://www.england.nhs.uk/statistics/statistical-work-areas/covid-19-daily-deaths/
today = datetime.date.today()

try:
    england_deaths_df = get_england_deaths_df(today)
except HTTPError:
    # today's data isn't out yet, so try getting yesterday's instead
    yesterday = today - datetime.timedelta(days=1)
    england_deaths_df = get_england_deaths_df(yesterday)

In [4]:
# data taken from PHE's tracker:
# https://www.arcgis.com/apps/opsdashboard/index.html#/f94c3c90da5b4e9f9a0b19484dd4bb14
url = "https://fingertips.phe.org.uk/documents/Historic%20COVID-19%20Dashboard%20Data.xlsx"
swni_deaths_df = pd.read_excel(
    url, 
    sheet_name="UK Deaths",
    header=7,
    index_col=0
)

swni_deaths_df = swni_deaths_df.fillna(method="bfill", axis=1)
swni_deaths_df[["Deaths", "UK"]] = swni_deaths_df[["Deaths", "UK"]].fillna(method="bfill")
swni_deaths_df = (
    swni_deaths_df
    .fillna(method="ffill", axis=1, limit=1)
    .fillna(0.)
    .astype(int)
    .drop(columns=["Deaths", "UK", "England"])
    .T
)
swni_deaths_df.insert(0, pd.to_datetime(datetime.date(2020, 3, 9)), 0)
deaths_df = pd.concat([swni_deaths_df, england_deaths_df])
deaths_df.index = deaths_df.index.rename("area_name")
columns_index = pd.MultiIndex.from_product([deaths_df.columns, ["deaths"]], names=[None, None])
deaths_df.columns = columns_index

In [5]:
# data taken from PHE's tracker:
# https://www.arcgis.com/apps/opsdashboard/index.html#/f94c3c90da5b4e9f9a0b19484dd4bb14
url = "https://fingertips.phe.org.uk/documents/Historic%20COVID-19%20Dashboard%20Data.xlsx"

england_regional_cases_df = pd.read_excel(
    url, 
    sheet_name="NHS Regions",
    header=7,
    index_col=1
)

country_cases_df = pd.read_excel(
    url,
    sheet_name="Countries",
    header=9,
    index_col=1,
)

country_cases_df.index = country_cases_df.index.rename("area_name").str.strip()
country_cases_df = country_cases_df.drop(index=["England", "UK"])

cases_df = pd.concat([england_regional_cases_df, country_cases_df])
cases_df = (
    cases_df
    .drop(index=[np.nan], columns=["Area Code"])
    .fillna(method="bfill", axis=1)
    .astype(int)
)
cases_df.index = cases_df.index.rename("area_name").str.strip()

index = pd.MultiIndex.from_product([cases_df.columns, ["confirmed_cases"]])
cases_df.columns = index

In [19]:
input_df = cases_df.merge(deaths_df, on="area_name").stack(level=1).fillna(method="ffill", axis=1).astype(int)
home_nations_data = input_df.loc[["England", "Scotland", "Wales", "Northern Ireland"]]
uk_confirmed_cases = home_nations_data.xs("confirmed_cases", level=1).sum()
uk_deaths = home_nations_data.xs("deaths", level=1).sum()
input_df.loc[("UK", "confirmed_cases"), :] = uk_confirmed_cases
input_df.loc[("UK", "deaths"), :] = uk_deaths
input_df = input_df.astype(int)
input_df

2020-03-09  2020-03-10  2020-03-11  \
area_name                                                                      
London                   confirmed_cases          61          91         104   
                         deaths                    4           4           8   
South East               confirmed_cases          43          51          60   
                         deaths                    5           6           7   
South West               confirmed_cases          38          41          44   
                         deaths                    0           0           1   
East of England          confirmed_cases          24          29          32   
                         deaths                    6           6           6   
Midlands                 confirmed_cases          31          36          42   
                         deaths                    5           5           6   
North East and Yorkshire confirmed_cases          22          24          32   
                         deaths                    0           0           0   
North West               confirmed_cases          35          37          43   
                         deaths                    2           2           2   
England                  confirmed_cases         280         324         387   
                         deaths                   22          23          30   
Scotland                 confirmed_cases          27          27          36   
                         deaths                    0           0           0   
Wales                    confirmed_cases           6           6          15   
                         deaths                    0           0           0   
Northern Ireland         confirmed_cases          16          16          18   
                         deaths                    0           0           0   
UK                       confirmed_cases         329         373         456   
                         deaths                   22          23          30   

                                          2020-03-12  2020-03-13  2020-03-14  \
area_name                                                                      
London                   confirmed_cases         136         167         313   
                         deaths                   10          16          28   
South East               confirmed_cases          83         110         144   
                         deaths                    7           8           9   
South West               confirmed_cases          42          49          53   
                         deaths                    1           1           3   
East of England          confirmed_cases          32          39          43   
                         deaths                    6           7           9   
Midlands                 confirmed_cases          44          59          75   
                         deaths                   11          16          20   
North East and Yorkshire confirmed_cases          44          49          67   
                         deaths                    0           0           0   
North West               confirmed_cases          53          62          69   
                         deaths                    4           6           7   
England                  confirmed_cases         491         645         925   
                         deaths                   39          54          76   
Scotland                 confirmed_cases          60          85         121   
                         deaths                    0           0           0   
Wales                    confirmed_cases          25          38          60   
                         deaths                    0           0           0   
Northern Ireland         confirmed_cases          19          29          34   
                         deaths                    0           0           0   
UK                       confirmed_cases 

In [20]:
# sort the rows by number of cases
input_df = input_df.sort_values(max(input_df.columns), ascending=False)

In [21]:
# NHS England population statistics taken from:
# https://www.ons.gov.uk/peoplepopulationandcommunity/populationandmigration/populationestimates/datasets/clinicalcommissioninggroupmidyearpopulationestimates
#
# Scotland, Wales and Northern Ireland population statistics taken from:
# https://www.ons.gov.uk/peoplepopulationandcommunity/populationandmigration/populationestimates
regional_populations = {
    "London": 8_908_081,
    "North West": 7_012_947,
    "North East and Yorkshire": 8_566_925,
    "Midlands": 10_537_679,
    "East of England": 6_493_188,
    "South East": 8_852_361,
    "South West": 5_605_997,
    "England": 55_977_178,
    "Wales": 3_136_400,
    "Scotland": 5_454_000,
    "Northern Ireland": 1_876_000,
}

countries = ["England", "Scotland", "Wales", "Northern Ireland"]
regional_populations["UK"] = sum(regional_populations[country] for country in countries)

In [59]:
fig = go.Figure()

areas = input_df.index.droplevel(1).unique()

confirmed_cases_traces = []
deaths_traces = []
prevalence_traces = []
case_fatality_rate_traces = []

# add a line to the figure for each area
for area in areas:
    area_data = input_df.T[[area]].reset_index().rename(columns={area: "measure", "index": "date"})
    area_data.columns = area_data.columns.rename(names=[None, None])
    
    area_data["measure", "delta_cases"] = area_data["measure", "confirmed_cases"].diff().fillna(0).astype(int)
    area_data["measure", "delta_deaths"] = area_data["measure", "deaths"].diff().fillna(0).astype(int)
    area_data["measure", "prevalence"] = area_data["measure", "confirmed_cases"] / regional_populations[area]
    area_data["measure", "case_fatality_rate"] = (
        area_data["measure", "deaths"] / area_data["measure", "confirmed_cases"]
    )
        
    area_data["text"] = (area_data["date"].dt.strftime("%d %B") + " | " + f"{area}<br>" \
                         "Cases: " + area_data["measure", "confirmed_cases"].apply("{:,}".format) + " (" + \
                         area_data["measure", "delta_cases"].apply("{:+,}".format)) + ")<br>" \
                         "Deaths: " + area_data["measure", "deaths"].apply("{:,}".format) + " (" + \
                         area_data["measure", "delta_deaths"].apply("{:+,}".format) + ")"
    
    confirmed_cases_traces.append(
        go.Scatter(
            x=area_data["date"],
            y=area_data["measure", "confirmed_cases"],
            name=area,
            # deselect the "England" and "UK" cases by default to avoid overshadowing other lines
            visible=True if area not in {"UK", "England"} else "legendonly",
            text=area_data["text"],
            hovertemplate="%{text}<extra></extra>"
        )
    )
    
    deaths_traces.append(
        go.Scatter(
            x=area_data["date"],
            y=area_data["measure", "deaths"],
            name=area,
            # hide all the deaths traces by default
            visible=False,
            text=area_data["text"],
            hovertemplate="%{text}<extra></extra>"
        )
    )
    
    prevalence_traces.append(
        go.Scatter(
            x=area_data["date"],
            y=area_data["measure", "prevalence"],
            name=area,
            # hide all the deaths traces by default
            visible=False,
            text=area_data["text"],
            hovertemplate="%{text}<extra></extra>"
        )
    )
    
    case_fatality_rate_traces.append(
        go.Scatter(
            x=area_data["date"],
            y=area_data["measure", "case_fatality_rate"],
            name=area,
            # hide all the deaths traces by default
            visible=False,
            text=area_data["text"],
            hovertemplate="%{text}<extra></extra>"
        )
    )
    
# add all the traces
for trace in confirmed_cases_traces + deaths_traces + prevalence_traces + case_fatality_rate_traces:
    fig.add_trace(trace)

# add a vertical line showing when the UK went into "lockdown"...
fig.add_shape(
    type="line",
    xref="x",
    yref="paper",
    x0=datetime.date(2020, 3, 23),
    y0=0,
    x1=datetime.date(2020, 3, 23),
    y1=0.9,
    opacity=0.8,
)    

# ...and then annotate it with some text
lockdown_annotation = dict(
    xref="x",
    yref="paper",
    x=datetime.date(2020, 3, 23),
    y=0.7,
    text="Lockdown commenced 23/03/2020",
    font_color="#000",
    showarrow=True,
    align="center",
    arrowhead=2,
    arrowsize=1,
    arrowwidth=1,
    arrowcolor="#636363",
    ax=-20,
    ay=0,
    xanchor="right",
    opacity=0.8,
)

total_cases = input_df.loc["UK", "confirmed_cases"].iloc[-1]
total_deaths = input_df.loc["UK", "deaths"].iloc[-1]
latest_prevalence = total_cases / regional_populations["UK"]
latest_case_fatality_rate = total_deaths / total_cases

total_cases_annotation = dict(
    xref="paper",
    yref="paper",
    x=0.05,
    y=0.95,
    text=f"Total UK Cases as of {input_df.columns[-1]: %-d %B %Y}:<br>" \
         f"{total_cases:,}",
    font_color="#000",
    showarrow=False,
    align="left"
)

total_deaths_annotation = dict(
    xref="paper",
    yref="paper",
    x=0.05,
    y=0.95,
    text=f"Total UK Deaths as of {input_df.columns[-1]: %-d %B %Y}:<br>" \
         f"{total_deaths:,}",
    font_color="#000",
    showarrow=False,
    align="left"
)

total_prevalence_annotation = dict(
    xref="paper",
    yref="paper",
    x=0.05,
    y=0.95,
    text=f"Total UK Period Prevalence as of {input_df.columns[-1]: %-d %B %Y}:<br>" \
         f"{latest_prevalence:.2%}",
    font_color="#000",
    showarrow=False,
    align="left"
)

total_case_fatality_annotation = dict(
    xref="paper",
    yref="paper",
    x=0.05,
    y=0.95,
    text=f"Total UK Case Fatality Rate as of {input_df.columns[-1]: %-d %B %Y}:<br>" \
         f"{latest_case_fatality_rate:.1%}",
    font_color="#000",
    showarrow=False,
    align="left"
)


fig.update_layout(
    updatemenus=[{
        "type": "buttons",
        "direction": "right",
        "x": 0.505,
        "y": 1.125,
        "buttons": list([
            {
                "label": "Cases",
                "method": "update",
                "args": [
                    {"visible": ["legendonly"] * 2 + [True] * (len(areas) - 2) + [False] * (len(areas) * 3)}, 
                    {
                        "title": "Confirmed Covid-19 Cases Per Region",
                        "yaxis": {"title": "Number of Confirmed Cases"},
                        "annotations": [lockdown_annotation] + [total_cases_annotation],
                    }
                ]
            },
            {
                "label": "Deaths",
                "method": "update",
                "args": [
                    {
                        "visible": [False] * len(areas) + ["legendonly"] * 2 + [True] * (len(areas) - 2) + [False] * (len(areas) * 2),
                    }, 
                    {
                        "title": "Covid-19 Deaths Per Region",
                        "yaxis": {"title": "Number of Deaths", "tickformat": ",d"},
                        "annotations": [lockdown_annotation] + [total_deaths_annotation],
                    }
                ]
            },
            {
                "label": "Prevalence",
                "method": "update",
                "args": [
                    {
                        "visible": [False] * len(areas) * 2 + [True] + ["legendonly"] * (len(areas) - 1) + [False] * len(areas)
                    },
                    {
                        "title": "Covid-19 Prevalence Per Region",
                        "yaxis": {"title": "Prevalence", "tickformat": ".2%"},
                        "annotations": [lockdown_annotation] + [total_prevalence_annotation],
                    }
                ],
            },
            {
                "label": "Case Fatality Rate",
                "method": "update",
                "args": [
                    {
                        "visible": [False] * len(areas) * 3 + [True] + ["legendonly"] * (len(areas) - 1)
                    },
                    {
                        "title": "Covid-19 Case Fatality Rate Per Region",
                        "yaxis": {"title": "Case Fatality Rate", "tickformat": ".1%"},
                        "annotations": [lockdown_annotation] + [total_case_fatality_annotation],
                    }
                ],
            }
        ])
    }],
    title={
        "text": "Confirmed Covid-19 Cases Per Region",
        "x": 0.45,
    },
    xaxis={
        "title": "Date",
        "tickformat": '%d %b',
        "tickangle": -45,
    },
    yaxis={
        "title": "Number of Confirmed Cases",
        "tickformat": ",d",
    },
    hovermode="closest",
    annotations=[lockdown_annotation, total_cases_annotation],
)
    
fig

In [62]:
fig = go.Figure(
    layout={
        "title": {
            "text": "Daily Number of New Covid-19 Cases Per Region",
            "x": 0.5
        },
        "xaxis": {
            "title": "Date",
            "tickformat": '%d %b',
            "tickangle": -45
        },
        "yaxis": {
            "title": "Daily Number of New Cases",
            "tickformat": ',d',
        },
        "legend": {
            "x": 0,
            "y": 1,
            "bgcolor": "RGBA(0,0,0,0)"
        },
        "hovermode": "closest"
    }
)

new_cases_traces = []
new_deaths_traces = []

areas = input_df.index.droplevel(1).unique()
for area in areas:
    area_data = input_df.T[[area]].reset_index().rename(columns={area: "measure", "index": "date"})
    area_data.columns = area_data.columns.rename(names=[None, None])
    
    area_data["measure", "new_cases"] = (
        area_data["measure", "confirmed_cases"].diff().fillna(0.).astype(int).clip(lower=0)
    )
    area_data["measure", "delta_new_cases"] = (
        area_data["measure", "new_cases"].diff().fillna(0).astype(int)
    )
    area_data["measure", "new_deaths"] = (
        area_data["measure", "deaths"].diff().fillna(0.).astype(int).clip(lower=0)
    )
    area_data["measure", "delta_new_deaths"] = (
        area_data["measure", "new_deaths"].diff().fillna(0).astype(int)
    )
        
    area_data["text"] = (
        area_data["date"].dt.strftime("%d %B") + " | " + f"{area}<br>" \
        "New cases: " + area_data["measure", "new_cases"].apply("{:,}".format) + \
        " (" + area_data["measure", "delta_new_cases"].apply("{:+,}".format) + ")<br>" \
        "New deaths: " + area_data["measure", "new_deaths"].apply("{:,}".format) + \
        " (" + area_data["measure", "delta_new_deaths"].apply("{:+,}".format) + ")"
    )
    
    new_cases_traces.append(
        go.Bar(
            x=area_data["date"],
            y=area_data["measure", "new_cases"],
            name=area,
            visible=True if area == "UK" else "legendonly",
            text=area_data["text"],
            hovertemplate="%{text}<extra></extra>"
        )
    )
        
    new_deaths_traces.append(
        go.Bar(
            x=area_data["date"],
            y=area_data["measure", "new_deaths"],
            name=area,
            visible=False,
            text=area_data["text"],
            hovertemplate="%{text}<extra></extra>"
        )
    )

for trace in new_cases_traces + new_deaths_traces:
    fig.add_trace(trace)

# add a vertical line showing when the UK went into "lockdown"...
fig.add_shape(
    type="line",
    xref="x",
    yref="paper",
    x0=datetime.date(2020, 3, 23),
    y0=0,
    x1=datetime.date(2020, 3, 23),
    y1=0.9,
    opacity=0.8,
)    

# ...and then annotate it with some text
fig.add_annotation(
    xref="x",
    yref="paper",
    x=datetime.date(2020, 3, 23),
    y=0.45,
    text="Lockdown commenced 23/03/2020",
    font_color="#000",
    showarrow=True,
    align="center",
    arrowhead=2,
    arrowsize=1,
    arrowwidth=1,
    arrowcolor="#636363",
    ax=-20,
    ay=0,
    xanchor="right",
    opacity=0.8,
)

fig.update_layout(
    updatemenus=[{
        "type": "buttons",
        "direction": "right",
        "x": 0.225,
        "y": 1.125,
        "buttons": list([
            {
                "label": "New Cases",
                "method": "update",
                "args": [
                    {
                        "visible": [True] + ["legendonly"] * (len(areas) - 1) + [False] * len(areas)
                    }, 
                    {
                        "title": "Daily Number of New Covid-19 Cases Per Region",
                        "yaxis": {"title": "Number of New Confirmed Cases", "tickformat": ",d"},
                    }
                ]
            },
            {
                "label": "New Deaths",
                "method": "update",
                "args": [
                    {
                        "visible": [False] * len(areas) + ["True"] + ["legendonly"] * (len(areas) - 1),
                    }, 
                    {
                        "title": "Daily Number of New Covid-19 Deaths Per Region",
                        "yaxis": {"title": "Number of New Deaths", "tickformat": ",d"},
                    }
                ]
            }
        ])
    }]
)
    
fig